# Build AI Agents with Vertex AI Agent Engine and Couchbase

[**Vertex AI Agent Engine**](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/) is a Google Cloud service that helps you build and scale AI agents in production. You can use the Agent Engine with **Couchbase Capella** and your preferred framework to build AI agents for a variety of use cases, including agentic **RAG** (Retrieval-Augmented Generation).

---

## 🛠️ Get Started

This tutorial demonstrates how to use the **Agent Engine with Couchbase** to build a RAG agent that can answer questions about sample data. It uses **Couchbase Vector Search** with **LangChain** to implement the retrieval tools for the agent.

---

## Prerequisites

Before you begin, ensure you have the following:

* A **Couchbase Capella cluster** in your preferred **region like Google Cloud**.
  To create a new cluster, see: [Create a Free Cluster in Couchbase Capella](https://docs.couchbase.com/cloud/get-started/create-account.html).

* A **Google Cloud project with Vertex AI enabled**.
  To set up a project, refer to: [Set up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

* **Setup up Google Vertex AI Agent Engine:**.
  [Follow these instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/set-up) to setup a Google Cloud project with billing enabled, have the required IAM permissions, enable the Vertex AI, Cloud Storage, Cloud Logging, Cloud Monitoring, and Cloud Trace APIs, set up a Cloud Storage bucket, and install the Vertex AI SDK for Python.

---

## Set up your Environment

Create an interactive Python notebook by saving a file with the `.ipynb` extension using **Google Colab**. This notebook will allow you to run Python code snippets individually and build your agent step by step.

---


# Setting the Stage: Installing Necessary Libraries

We'll install the following key libraries:
- `datasets`: For loading and managing our training data
- `langchain-couchbase`: To integrate Couchbase with LangChain for vector storage and caching
- `langchain-google-vertexai`: For accessing Google VertexAI's embedding and chat models
- `python-dotenv`: For securely managing environment variables and API keys

These libraries provide the foundation for building a semantic search engine with vector embeddings,
database integration, and agent-based RAG capabilities.

In [ ]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,agent_engines]" requests datasets couchbase langchain langchain-community langchain-couchbase langchain-google-vertexai google-cloud-aiplatform langchain_google_genai requests beautifulsoup4 langsmith

# Importing Necessary Libraries
The script starts by importing a series of libraries required for various tasks, including handling JSON, logging, time tracking, Couchbase connections, embedding generation, and dataset loading.

In [ ]:
import getpass
import json
import logging
import os
import time
from datetime import timedelta

from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.diagnostics import PingState, ServiceType
from couchbase.exceptions import (InternalServerFailureException,
                                  QueryIndexAlreadyExistsException,
                                  ServiceUnavailableException)
from couchbase.management.buckets import CreateBucketSettings
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions
from datasets import load_dataset
from dotenv import load_dotenv
from datasets import load_dataset

# Loading Sensitive Information
In this section, we prompt the user to input essential configuration settings needed. These settings include sensitive information like database credentials, and specific configuration names. Instead of hardcoding these details into the script, we request the user to provide them at runtime, ensuring flexibility and security.

The script uses environment variables to store sensitive information, enhancing the overall security and maintainability of your code by avoiding hardcoded values.

In [ ]:
# Load environment variables
load_dotenv()

CB_HOST = os.getenv('CB_HOST') or input("Enter Couchbase host (default: couchbase://localhost): ") or 'couchbase://localhost'
CB_USERNAME = os.getenv('CB_USERNAME') or input("Enter Couchbase username (default: Administrator): ") or 'Administrator'
CB_PASSWORD = os.getenv('CB_PASSWORD') or getpass.getpass("Enter Couchbase password (default: password): ") or 'password'
CB_BUCKET_NAME = os.getenv('CB_BUCKET_NAME') or input("Enter bucket name (default: gemini): ") or 'gemini'
INDEX_NAME = os.getenv('INDEX_NAME') or input("Enter index name (default: vector_search_gemini): ") or 'vector_search_gemini'
SCOPE_NAME = os.getenv('SCOPE_NAME') or input("Enter scope name (default: _default): ") or '_default'
COLLECTION_NAME = os.getenv('COLLECTION_NAME') or input("Enter collection name (default: _default): ") or '_default'

print("Configuration loaded successfully")

# Connecting to the Couchbase Cluster
Connecting to a Couchbase cluster is the foundation of our project. Couchbase will serve as our primary data store, handling all the storage and retrieval operations required for our semantic search engine. By establishing this connection, we enable our application to interact with the database, allowing us to perform operations such as storing embeddings, querying data, and managing collections. This connection is the gateway through which all data will flow, so ensuring it's set up correctly is paramount.

In [ ]:
# Connect to Couchbase
try:
    auth = PasswordAuthenticator(CB_USERNAME, CB_PASSWORD)
    options = ClusterOptions(auth)
    cluster = Cluster(CB_HOST, options)
    cluster.wait_until_ready(timedelta(seconds=5))
    print("Successfully connected to Couchbase")
except Exception as e:
    print(f"Failed to connect to Couchbase: {str(e)}")
    raise

## Setting Up Collections in Couchbase

The setup_collection() function handles creating and configuring the hierarchical data organization in Couchbase:

1. Bucket Creation:
   - Checks if specified bucket exists, creates it if not
   - Sets bucket properties like RAM quota (1024MB) and replication (disabled)
   - Note: If you are using Capella, create a bucket manually called vector-search-testing(or any name you prefer) with the same properties.

2. Scope Management:  
   - Verifies if requested scope exists within bucket
   - Creates new scope if needed (unless it's the default "_default" scope)

3. Collection Setup:
   - Checks for collection existence within scope
   - Creates collection if it doesn't exist
   - Waits 2 seconds for collection to be ready

Additional Tasks:
- Creates primary index on collection for query performance
- Clears any existing documents for clean state
- Implements comprehensive error handling and logging

The function is called twice to set up:
1. Main collection for vector embeddings
2. Cache collection for storing results


In [ ]:
def setup_collection(cluster, bucket_name, scope_name, collection_name):
    try:
        # Check if bucket exists, create if it doesn't
        try:
            bucket = cluster.bucket(bucket_name)
            logging.info(f"Bucket '{bucket_name}' exists.")
        except Exception as e:
            logging.info(f"Bucket '{bucket_name}' does not exist. Creating it...")
            bucket_settings = CreateBucketSettings(
                name=bucket_name,
                bucket_type='couchbase',
                ram_quota_mb=1024,
                flush_enabled=True,
                num_replicas=0
            )
            cluster.buckets().create_bucket(bucket_settings)
            bucket = cluster.bucket(bucket_name)
            logging.info(f"Bucket '{bucket_name}' created successfully.")

        bucket_manager = bucket.collections()

        # Check if scope exists, create if it doesn't
        scopes = bucket_manager.get_all_scopes()
        scope_exists = any(scope.name == scope_name for scope in scopes)

        if not scope_exists and scope_name != "_default":
            logging.info(f"Scope '{scope_name}' does not exist. Creating it...")
            bucket_manager.create_scope(scope_name)
            logging.info(f"Scope '{scope_name}' created successfully.")

        # Check if collection exists, create if it doesn't
        collections = bucket_manager.get_all_scopes()
        collection_exists = any(
            scope.name == scope_name and collection_name in [col.name for col in scope.collections]
            for scope in collections
        )

        if not collection_exists:
            logging.info(f"Collection '{collection_name}' does not exist. Creating it...")
            bucket_manager.create_collection(scope_name, collection_name)
            logging.info(f"Collection '{collection_name}' created successfully.")
        else:
            logging.info(f"Collection '{collection_name}' already exists. Skipping creation.")

        # Wait for collection to be ready
        collection = bucket.scope(scope_name).collection(collection_name)
        time.sleep(2)  # Give the collection time to be ready for queries

        # Ensure primary index exists
        try:
            cluster.query(f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{bucket_name}`.`{scope_name}`.`{collection_name}`").execute()
            logging.info("Primary index present or created successfully.")
        except Exception as e:
            logging.warning(f"Error creating primary index: {str(e)}")

        # Clear all documents in the collection
        try:
            query = f"DELETE FROM `{bucket_name}`.`{scope_name}`.`{collection_name}`"
            cluster.query(query).execute()
            logging.info("All documents cleared from the collection.")
        except Exception as e:
            logging.warning(f"Error while clearing documents: {str(e)}. The collection might be empty.")

        return collection
    except Exception as e:
        raise RuntimeError(f"Error setting up collection: {str(e)}")

setup_collection(cluster, CB_BUCKET_NAME, SCOPE_NAME, COLLECTION_NAME)


# Configuring and Initializing Couchbase Vector Search Index for Semantic Document Retrieval

Semantic search requires an efficient way to retrieve relevant documents based on a user's query. This is where the Couchbase Vector Search Index comes into play. In this step, we load the Vector Search Index definition from a JSON file, which specifies how the index should be structured. This includes the fields to be indexed, the dimensions of the vectors, and other parameters that determine how the search engine processes queries based on vector similarity.

This vector search index configuration requires specific default settings to function properly. This workshop uses the bucket named `gemini` with the scope `_default` and collection `_default`. The configuration is set up for vectors with exactly `768 dimensions`, using `dot product` similarity and optimized for `recall`. If you want to use a different bucket, scope, or collection, you will need to modify the index configuration accordingly.

For more information on creating a vector search index, please follow the instructions at [Couchbase Vector Search Documentation](https://docs.couchbase.com/cloud/vector-search/create-vector-search-index-ui.html).

In [ ]:
# Load index definition
try:
    with open('index.json', 'r') as file:
        index_definition = json.load(file)
except FileNotFoundError as e:
    print(f"Error: crew_index.json file not found: {str(e)}")
    raise
except json.JSONDecodeError as e:
    print(f"Error: Invalid JSON in crew_index.json: {str(e)}")
    raise
except Exception as e:
    print(f"Error loading index definition: {str(e)}")
    raise

# Creating or Updating Search Indexes

With the index definition loaded, the next step is to create or update the **Vector Search Index** in Couchbase. This step is crucial because it optimizes our database for vector similarity search operations, allowing us to perform searches based on the semantic content of documents rather than just keywords. By creating or updating a Vector Search Index, we enable our search engine to handle complex queries that involve finding semantically similar documents using vector embeddings, which is essential for a robust semantic search engine.

In [ ]:
try:
    scope_index_manager = cluster.bucket(CB_BUCKET_NAME).scope(SCOPE_NAME).search_indexes()

    # Check if index already exists
    existing_indexes = scope_index_manager.get_all_indexes()
    index_name = index_definition["name"]

    if index_name in [index.name for index in existing_indexes]:
        logging.info(f"Index '{index_name}' found")
    else:
        logging.info(f"Creating new index '{index_name}'...")

    # Create SearchIndex object from JSON definition
    search_index = SearchIndex.from_json(index_definition)

    # Upsert the index (create if not exists, update if exists)
    scope_index_manager.upsert_index(search_index)
    logging.info(f"Index '{index_name}' successfully created/updated.")

except QueryIndexAlreadyExistsException:
    logging.info(f"Index '{index_name}' already exists. Skipping creation/update.")
except ServiceUnavailableException:
    raise RuntimeError("Search service is not available. Please ensure the Search service is enabled in your Couchbase cluster.")
except InternalServerFailureException as e:
    logging.error(f"Internal server error: {str(e)}")
    raise

# Initialize the Vertex AI SDK

Run the following code in your notebook, replacing the placeholder values with your Google Cloud project ID, region, and staging bucket:

In [ ]:
PROJECT_ID = "your_vertex_project_ID"  # Replace with your project ID
LOCATION = "your_region"         # Replace with your preferred region, e.g. "us-central1"
STAGING_BUCKET = "gs://your_bucket_name"  # Replace with your bucket

import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

# Ingest Data into Couchbase
Run the following code to scrape sample data from Wikipedia about Star Warss, convert the text into vector embeddings using the text-embedding-005 model, and then store this data in the corresponding collections in Couchbase
.

In [ ]:
import requests
from bs4 import BeautifulSoup
import certifi
import os
from vertexai.language_models import TextEmbeddingModel
import uuid

from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster, ClusterOptions
from couchbase.collection import InsertOptions
from couchbase.exceptions import CouchbaseException
from datetime import timedelta  # ✅ Required for wait_until_ready

# Scrape website content
def scrape_website(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    content = ' '.join([p.text for p in soup.find_all('p')])
    return content

# Split into chunks
def split_into_chunks(text, chunk_size=1000):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Get embeddings using GCP
def get_text_embeddings(chunks):
    model = TextEmbeddingModel.from_pretrained("text-embedding-005")
    embeddings = model.get_embeddings(chunks)
    return [embedding.values for embedding in embeddings]

# Store in Couchbase
def write_to_couchbase(embeddings, chunks):
    try:
        cluster = Cluster(CB_HOST, ClusterOptions(
            PasswordAuthenticator(CB_USERNAME, CB_PASSWORD)))
        cluster.wait_until_ready(timedelta(seconds=10))  # ✅ Correct usage
        bucket = cluster.bucket(CB_BUCKET_NAME)
        collection = bucket.scope(SCOPE_NAME).collection(COLLECTION_NAME)

        for i in range(len(chunks)):
            doc_id = f"{COLLECTION_NAME}_{uuid.uuid4()}"
            doc = {
                "text": chunks[i],
                "embedding": embeddings[i]
            }
            collection.insert(doc_id, doc, InsertOptions(timeout=timedelta(seconds=5)))
            print(f"📥 Inserted document ID: {doc_id}")

    except CouchbaseException as e:
        print(f"❌ Error inserting into Couchbase: {e}")

# Ingest content from a URL
def ingest_url(url, label):
    print(f"\n🔍 Processing: {label}")
    content = scrape_website(url)
    chunks = split_into_chunks(content)
    embeddings = get_text_embeddings(chunks)
    write_to_couchbase(embeddings, chunks)

# Run ingestion for Wikipedia sources
ingest_url("https://en.wikipedia.org/wiki/Star_Wars", "Star Wars")

# Create the Agent

In this section, you define tools that the agent can use to query your collections using Couchase Vector Search, create a memory system to maintain conversation context, and then initialize the agent using LangChain.

## Define tools for the agent

### 1. Star Wars Query Tool

Run the following code to create a tool that uses Couchbase Vector Search to query the starwars data from the _default collection:

In [ ]:
load_dotenv()

def star_wars_query_tool(
    query: str
):
    """
    Retrieves vectors from a Couchbase database and uses them to answer a question related to Star wars.

    Args:
        query: The question to be answered about star wars.

    Returns:
        A dictionary containing the response to the question.
    """
    from langchain.chains import ConversationalRetrievalChain, RetrievalQA
    from langchain_couchbase import CouchbaseVectorStore
    from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
    from langchain_google_vertexai import VertexAIEmbeddings, ChatVertexAI
    from langchain.prompts import PromptTemplate
    from couchbase.auth import PasswordAuthenticator
    from couchbase.cluster import Cluster
    from couchbase.options import ClusterOptions
    from datetime import timedelta
    import os

    # Create local Couchbase connection to avoid serialization issues
    auth = PasswordAuthenticator(
        os.getenv('CB_USERNAME', 'pdf'),
        os.getenv('CB_PASSWORD', 'Pdf1234!')
    )
    options = ClusterOptions(auth)
    local_cluster = Cluster(os.getenv('CB_HOST', 'couchbases://cb.hxdufdz4y5boruny.cloud.couchbase.com'), options)
    local_cluster.wait_until_ready(timedelta(seconds=5))


    embeddings = VertexAIEmbeddings(model_name="text-embedding-005")

    prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Do not return any answers from your own knowledge. Respond only in 2 or 3 sentences.

    {context}

    Question: {question}

     Also add the {context} to the response.
    """
    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    vs = CouchbaseVectorStore(
        cluster=local_cluster,
        bucket_name=os.getenv('CB_BUCKET_NAME', 'gemini'),
        scope_name=os.getenv('SCOPE_NAME', '_default'),
        collection_name=os.getenv('COLLECTION_NAME', '_default'),
        embedding=embeddings,
        index_name=os.getenv('INDEX_NAME', 'gemini_vector'),
        text_key="text",  # Use "text" since that's what you stored in your documents
    )
    llm = ChatVertexAI(
        model_name="gemini-2.5-pro",
        convert_system_message_to_human=True,
        max_output_tokens=1000,
        temperature=0.1,
    )
    retriever = vs.as_retriever()

    memory = ConversationBufferWindowMemory(
        memory_key="chat_history", k=5, return_messages=True
    )
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        combine_docs_chain_kwargs={"prompt": PROMPT},
    )
    response = conversation_chain({"question": query})

    return response

### 2. Blog Writing Tool

Run the following code to create a tool that writes a well structured blog post.

   - Takes the research summary as input
   - Structures and formats the information
   - Creates a polished, user-friendly response
   - Ensures proper attribution and citation



In [ ]:
def blog_writing_tool(
    research_context: str,
    topic: str = "Star Wars",
    blog_style: str = "informative"
):
    """
    Creates a well-structured blog post based on research context about Star Wars.

    Args:
        research_context: The research information to base the blog post on
        topic: The main topic for the blog post (default: "Star Wars")
        blog_style: The style of the blog post (informative, casual, academic, etc.)

    Returns:
        A dictionary containing the generated blog post
    """
    from langchain_google_vertexai import ChatVertexAI
    from langchain.prompts import PromptTemplate

    # Blog writing prompt template
    blog_prompt_template = """You are an expert blog writer specializing in {topic} content.
    Based on the research context provided, write a well-structured, engaging blog post.

    Research Context:
    {research_context}

    Please write a blog post with the following requirements:
    - Style: {blog_style}
    - Include a compelling title
    - Have clear sections with headings
    - Be approximately 500-800 words
    - Include an engaging introduction and conclusion
    - Use the research context as the primary source of information
    - Make it SEO-friendly with relevant keywords
    - Write in a {blog_style} tone

    Format the output as:
    # [Blog Title]

    ## Introduction
    [Introduction content]

    ## [Section 1 Heading]
    [Section 1 content]

    ## [Section 2 Heading]
    [Section 2 content]

    ## Conclusion
    [Conclusion content]

    Blog Post:"""

    BLOG_PROMPT = PromptTemplate(
        template=blog_prompt_template,
        input_variables=["research_context", "topic", "blog_style"]
    )

    # Initialize the LLM for blog writing
    llm = ChatVertexAI(
        model_name="gemini-2.5-pro",
        convert_system_message_to_human=True,
        max_output_tokens=2000,
        temperature=0.7,  # Higher temperature for more creative writing
    )

    # Generate the blog post
    formatted_prompt = BLOG_PROMPT.format(
        research_context=research_context,
        topic=topic,
        blog_style=blog_style
    )

    response = llm.invoke(formatted_prompt)

    return {
        "blog_post": response.content,
        "topic": topic,
        "style": blog_style,
        "word_count": len(response.content.split())
    }


## Create a memory system

You can use LangChain to create memory for your agent so that it can maintain conversation context across multiple prompts:

In [ ]:
from langchain.memory import ChatMessageHistory

# Initialize session history
store = {}

def get_session_history(session_id: str):
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]

## Initialize the agent

Create the agent using LangChain. This agent uses the tools and memory system that you defined.

In [ ]:
from vertexai.preview.reasoning_engines import LangchainAgent

unified_multi_agent = LangchainAgent(
    model="gemini-2.5-pro",
    chat_history=get_session_history,
    model_kwargs={"temperature": 0.3},  # Balanced temperature
    tools=[
        star_wars_query_tool,          # Research tool
        blog_writing_tool,             # Blog writing tool
    ],
    agent_executor_kwargs={"return_intermediate_steps": True},
    system_instruction="""You are a multi-agent AI system specialized in Star Wars content research and blog creation.

You have access to two main tools that you can chain together:
1. star_wars_query_tool: For researching Star Wars topics using vector database
2. blog_writing_tool: For creating well-structured blog posts from research content

🔗 TOOL CHAINING INSTRUCTIONS:

FOR BLOG POST REQUESTS:
When a user asks for a blog post about a Star Wars topic, follow these steps:
1. FIRST: Use star_wars_query_tool to research the topic and gather context
2. THEN: Use blog_writing_tool with the research context from step 1 as the 'research_context' parameter

FOR RESEARCH-ONLY REQUESTS:
When a user asks for information/research only, use star_wars_query_tool directly.

FOR BLOG WITH PROVIDED CONTEXT:
When a user provides their own research/context, use blog_writing_tool directly.

🎯 EXAMPLE WORKFLOW FOR BLOG CREATION:
User: "Write a blog post about Darth Vader"
1. Call star_wars_query_tool("Darth Vader journey character development")
2. Take the 'answer' from the result
3. Call blog_writing_tool(research_context=<answer from step 2>, topic="Darth Vader", blog_style="informative")
4. Return the generated blog post

Always:
- Provide detailed, accurate, and engaging responses
- Use the research context as the primary source for blog posts
- Maintain professional quality in all outputs
- Show your reasoning process when chaining tools"""
)

To test the agent with a sample query:

In [ ]:
from IPython.display import display, Markdown

# Test the unified agent with different use cases
test_cases = [
    {
        "query": "Create a complete blog post about Darth Vader's journey from Anakin Skywalker",
        "description": "Complete blog creation (research + writing)"
    },
    {
        "query": "While filming the original 1977 film, What was the amount of pay cut from his salary that George Lucas decided to takeas director in exchange for full ownership of the franchise's merchandising rights.",
        "description": "Research only"
    }
]

for i, test_case in enumerate(test_cases, 1):
    print(f"\n📋 TEST CASE {i}: {test_case['description']}")
    print(f"🔍 Query: {test_case['query']}")
    print("-" * 40)

    try:
        response = unified_multi_agent.query(
            input=test_case['query'],
            config={"configurable": {"session_id": f"unified_test_{i}"}}
        )

        output = response.get("output", "")
        print(f"Response length: {len(output)} characters")
        print(f" Response preview: {output[:200]}...")

        # Display full response if it's a blog post
        if "blog" in test_case['query'].lower():
            display(Markdown("### Generated Blog Post:"))
            display(Markdown(output))

    except Exception as e:
        print(f" Error: {str(e)}")



# Deploy the Agent
In this section, you deploy your agent to the Vertex AI Agent Engine as a managed service. This allows you to scale your agent and use it in production without managing the underlying infrastructure.

## Deploy your agent.
Run the following code to configure and deploy the agent in the Vertex AI Agent Engine:

In [ ]:
from vertexai import agent_engines

try:
    # Deploy the two-tool chained multi-agent system
    deployed_multi_agent = agent_engines.create(
        unified_multi_agent,
        requirements=[
            "google-cloud-aiplatform[agent_engines,langchain]",
            "cloudpickle==3.0.0",
            "pydantic>=2.10",
            "requests",
            "langchain-couchbase",
            "pymongo",
            "langchain-google-vertexai",
            "beautifulsoup4",  # For web scraping in research
        ],
        display_name="StarWars-TwoTool-ChainedAgent",
        description="Two-tool chained agent for Star Wars research and blog writing - automatically chains research and writing tools"
    )

    print("Two-Tool Chained Agent deployed successfully!")
    print(f"📍 Resource Name: {deployed_multi_agent.resource_name}")

except Exception as e:
    print(f"Deployment failed: {str(e)}")


## Retrieve the project URL.
Run the following code to retrieve the project number associated with your project ID. This project number will be used to construct the complete resource name for your deployed agent:

In [ ]:
from googleapiclient import discovery
from IPython.display import display, Markdown

# Retrieve the project number associated with your project ID
service = discovery.build("cloudresourcemanager", "v1")
request = service.projects().get(projectId=PROJECT_ID)
response = request.execute()
project_number = response["projectNumber"]
print(f"Project Number: {project_number}")
# The deployment creates a unique ID for your agent that you can find in the output

## Test the agent.
Run the following code to use your agent. Replace the placeholder with your agent's full resource name:

In [ ]:
from vertexai.preview import reasoning_engines

# Replace with your agent's full resource name from the previous step
REASONING_ENGINE_RESOURCE_NAME = "<resource-name>"

remote_agent = reasoning_engines.ReasoningEngine(REASONING_ENGINE_RESOURCE_NAME)

response = remote_agent.query(
    input="While filming the original 1977 film, What was the amount of pay cut from his salary that George Lucas decided to takeas director in exchange for full ownership of the franchise's merchandising rights.",
    config={"configurable": {"session_id": "demo"}},
)
print(response["output"])


Performing another test to write a blog with the remote deployed agent

In [ ]:
# Single Test Case: Blog Writing with Remote Unified Agent

from vertexai.preview import reasoning_engines
from IPython.display import display, Markdown

# Use the deployed unified multi-agent system
UNIFIED_AGENT_RESOURCE_NAME = "<resource-name>"

remote_unified_agent = reasoning_engines.ReasoningEngine(UNIFIED_AGENT_RESOURCE_NAME)

print("📝 Testing Blog Creation with Remote Unified Agent")
print("=" * 55)

# Single focused test case for blog writing
blog_query = "While filming the original 1977 film, What was the amount of pay cut from his salary that George Lucas decided to takeas director in exchange for full ownership of the franchise's merchandising rights."

print(f"🔍 Query: {blog_query}")
print(f"🎯 Expected: Complete blog post with research and writing")
print("-" * 55)

try:
    # Execute the blog creation request
    response = remote_unified_agent.query(
        input=blog_query,
        config={"configurable": {"session_id": "blog_test"}}
    )

    output = response.get("output", "")

    print(f"✅ Blog post generated: {len(output)} characters")

    # Display the generated blog post
    if output:
        print(f"📰 Blog Post Preview: {output[:200]}...")
        print("\n" + "="*55)
        print("📖 FULL BLOG POST:")
        print("="*55)
        display(Markdown(output))
    else:
        print("❌ No content generated")

except Exception as e:
    print(f"❌ Error: {str(e)}")

print("\n🎉 Blog creation test completed!")


# Next Steps
You can also debug and optimize your agents by enabling [tracing](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/manage/tracing) in the Agent Engine. Refer to the [Vertex AI Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/) documentation for other features and examples.
